## Chapter 20: Concurrent Executers



In [2]:
## flags.py

import time
from pathlib import Path
from typing import Callable, List

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')

def save_flag(img: bytes, filename: str) -> None:
    (DEST_DIR/ filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = httpx.get(url, timeout=6.1, follow_redirects=True)
    resp.raise_for_status()
    return resp.content

def download_many(cc_list: List[str]) -> int:
    for cc in sorted(cc_list):
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end= ' ', flush=True)
    return len(cc_list)


def main(downloader: Callable[[list[str]], int]) -> None:
    DEST_DIR.mkdir(exist_ok=True)
    t0 = time.perf_counter()
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')


if __name__ == '__main__':
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 downloads in 35.22s


In [3]:
from concurrent import futures

def download_one(cc: str) -> str:
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

def download_many(cc_list: List[str]) -> int:
    with futures.ThreadPoolExecutor() as executers:
        res = executers.map(download_one, sorted(cc_list))
    
    return len(list(res))


if __name__ == '__main__': 
    main(download_many)

FR ID ET DE CN NG BD JP US CD MX VN IN PK BR PH EG RU IR TR 
20 downloads in 4.14s


### What are `futures`?

**future**: It is used in asyncio, concurrent.futures and standard library. An instance of future represents a deferred computation that may or may not have completed.

Futures encapsulate pending operations so that we can put them in queues, check whether they are done, and retrieve results (or exceptions) when they become available. 

**IMPORTANT**: We don't make an instance future ourselves. this should be handled by framework. Why? A future is an instance of an operation that will eventually be run.therefore it must be scheduled to run, and that’s the job of the framework.

You submit a task to a thread or process pool. You can:

Call `.done()` repeatedly to check if it’s finished (polling, which is inefficient), or...

Use `.add_done_callback()` to register a function that should be automatically called once the task is done.


##### `.result()` vs. `.done()`

🔹 `.done()` — "Is it finished yet?"
- Returns a boolean: True if the task has finished, False otherwise.

- Non-blocking — it gives you an immediate answer.

- It does not return the result of the task.

----

🔸 `.result()` — "Give me the result!"
- Returns the actual result of the function you submitted.

- Blocking — if the task is not done yet, .result() waits until it is.

- Raises an exception if the function raised one during execution.

##### Running Futures ( `executor.map()` vs. `futures.as_completed()`)

`map` function is like the map in lists. the behavior is as follows:
- Submits all tasks at once.

- Blocks until all tasks are done.

- You don’t get access to individual Future objects.

- Results are returned in **input order**, regardless of when each finishes.

`as_completed()` takes a list of `futures` and yields them as they complete in **any order**.

In [4]:
from concurrent.futures import ThreadPoolExecutor
import time

def slow_square(n):
    time.sleep(n)
    return n * n

with ThreadPoolExecutor() as executor:
    results = executor.map(slow_square, list(range(10, 0, -1)))

for r in results:
    print(r)


100
81
64
49
36
25
16
9
4
1


In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def slow_square(n):
    time.sleep(n)
    return n * n

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(slow_square, n) for n in range(10, 0, -1)]

    for future in as_completed(futures):
        print(future.result())

1
4
9
16
25
36
49
64
81
100


**IMPORTANT**: processes are better for CPU-intensive tasks while threads and async programs are suitable for IO-bound tasks. Because processes have their own pyhton interpreter and memory but threads share the memory.